In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score
import joblib
import os

def train_model():
    print("Iniciando Protocolo de Entrenamiento - Proyecto Wow...")
    
    # --- RUTA CORREGIDA ---
    input_path = "../data/nhanes_labeled_data.csv"
    try:
        df = pd.read_csv(input_path)
    except FileNotFoundError:
        print(f"Error: Falta '{input_path}'")
        return

    # 2. Preprocesamiento
    X = df.drop(columns=['Diagnosis', 'ID']) 
    y_raw = df['Diagnosis']
    
    le = LabelEncoder()
    y = le.fit_transform(y_raw)
    
    # --- RUTA CORREGIDA ---
    if not os.path.exists("../models"):
        os.makedirs("../models")
    joblib.dump(le, '../models/label_encoder.joblib')
    print(f"Clases detectadas: {le.classes_}")

    # 3. Configuración del Modelo
    # Eliminamos 'use_label_encoder' para evitar el Warning
    model = xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=len(le.classes_),
        eval_metric='mlogloss',
        random_state=42,
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1
    )

    # 4. Validación Cruzada
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = []
    
    print("\nComenzando Stratified K-Fold (5 Folds)...")
    print("-" * 60)
    
    fold = 1
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        f1 = f1_score(y_test, y_pred, average='weighted')
        acc = accuracy_score(y_test, y_pred)
        f1_scores.append(f1)
        
        print(f"🔹 Fold {fold}: Accuracy={acc:.4f} | F1-Weighted={f1:.4f}")
        fold += 1

    print("-" * 60)
    print(f"Promedio F1-Weighted: {np.mean(f1_scores):.4f}")
    
    # 5. Entrenamiento Final
    print("\nEntrenando modelo final con TODOS los datos...")
    model.fit(X, y)
    
    # --- RUTA CORREGIDA ---
    model.save_model("../models/xgboost_clinical_v1.json")
    print("💾 Modelo guardado en '../models/xgboost_clinical_v1.json'")
    
    final_preds = model.predict(X)
    print("\n🔍 Reporte de Clasificación (Training Set - Sanity Check):")
    print(classification_report(y, final_preds, target_names=le.classes_))

if __name__ == "__main__":
    train_model()

Iniciando Protocolo de Entrenamiento - Proyecto Wow...
Clases detectadas: ['Anemia' 'Infection' 'Normal' 'Thrombocytopenia']

Comenzando Stratified K-Fold (5 Folds)...
------------------------------------------------------------


c:\Users\matir\ml_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:02:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🔹 Fold 1: Accuracy=0.9987 | F1-Weighted=0.9987


c:\Users\matir\ml_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:02:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🔹 Fold 2: Accuracy=0.9987 | F1-Weighted=0.9987


...

🔹 Fold 4: Accuracy=0.9980 | F1-Weighted=0.9980


c:\Users\matir\ml_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:02:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🔹 Fold 5: Accuracy=0.9967 | F1-Weighted=0.9967
------------------------------------------------------------
Promedio F1-Weighted: 0.9980 (+/- 0.0007)

Entrenando modelo final con TODOS los datos...


c:\Users\matir\ml_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:02:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


💾 Modelo guardado como 'xgboost_clinical_v1.json'

🔍 Reporte de Clasificación (Training Set - Sanity Check):
                  precision    recall  f1-score   support

          Anemia       1.00      1.00      1.00      1060
       Infection       1.00      1.00      1.00       440
          Normal       1.00      1.00      1.00      5792
Thrombocytopenia       1.00      1.00      1.00       235

        accuracy                           1.00      7527
       macro avg       1.00      1.00      1.00      7527
    weighted avg       1.00      1.00      1.00      7527

Pipeline finalizado. Listo para Fase 4 (Explicabilidad).
